In [ ]:
# import data
r05_urls = [
    "r05/angeles",
    "r05/klamath", 
    "r05/cleveland",
    "r05/eldorado",
    "r05/sequoia",
    "r05/inyo",
    "r05/klamath",
    "r05/laketahoebasin",
    "r05/lassen",
    "r05/lospadres",
    "r05/mendocino",
    "r05/modoc",
    "r05/plumas",
    "r05/sanbernardino",
    "r05/shasta-trinity",
    "r05/sierra",
    "r05/sixrivers",
    "r05/stanislaus",
    "r05/tahoe"
]

r04_urls = [
    "r04/humboldt-toiyabe"
]

r06_urls = [
    "r06/rogue-siskiyou"
]
all_urls = r05_urls + r04_urls + r06_urls

names = []
half = len(all_urls) // 2
for i, url in enumerate(all_urls[:half]):
    names[i] = all_urls[i].split("/")[-1]
    
